### Project Structure 
### twin_classification/
### ├── data/                  # Input images
### │   ├── you/
### │   └── twin/
### ├── models/                # Saved trained models
### │   ├── random_forest.pkl
### │   └── cnn_model.h5
### ├── notebooks/             # Notebooks for experimentation
### │   └── exploration.ipynb
### ├── src/                   # Python scripts
### │   ├── preprocess.py
### │   ├── train.py
### │   ├── evaluate.py
### │   └── predict.py
### ├── web/                   # Optional web interface
### │   ├── index.html
### │   ├── style.css
### │   └── app.py
### ├── requirements.txt       # Project dependencies
### └── README.md              # This file


# Twin Image Classifier Project 
**Goal:** Build a simple machine learning classifier that can tell the difference between
me and my identical twin using basic ML concepts (no deep learning).

## Step 1: Project Overview
- **Type:** Classification
- **Input: (Features)** Images of me and my twin
- **Output: (Targets)** "Neema" or "Amani"
- **Algorithms:** Decision Tree, Logistic Regression, KNN
- **Evaluation Metric:** Accuracy

# Twin Classification Project Notebook
# Author: Neema Kanyi
# Date: 2025-12-08

## BACKGROUND - Why I chose what 
### WHAT IS RANDOM FOREST CLASSIFIER?
The reason I went with Random Forest rather than KNN or SVM is because of efficiency